In [1]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
from random import shuffle
import copy

data = pd.read_csv('/home/kai/data/shiyi/data/avazu-ctr-prediction/train', nrows=1000000)

features = [3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23]

In [29]:
msk = np.random.rand(len(data)) < 0.8
X_train = data[msk].iloc[:,features]
X_test = data[~msk].iloc[:,features]
y_train = data[msk].iloc[:,1]
y_test = data[~msk].iloc[:,1]

In [27]:
size = 6
window = 8

In [12]:
print(list(data.columns))

['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


In [6]:
x_w2v = copy.deepcopy(data.iloc[:,features])

In [7]:
type(x_w2v)

pandas.core.frame.DataFrame

In [9]:
names = list(x_w2v.columns.values)
print(names)

['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


In [13]:
for i in names:
    x_w2v[i]=x_w2v[i].astype('category')
    x_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in x_w2v[i].cat.categories]

In [23]:
x_w2v = x_w2v.values.tolist()

In [24]:
for i in x_w2v:
    shuffle(i)

In [22]:
for col in x_w2v.columns:
    print(x_w2v[col].nunique())

7
7
2075
2030
21
2309
156
23
83431
4581
4
4
606
8
9
162
4
41
161
35


In [28]:
w2v = Word2Vec(x_w2v,size=size,window=window)

In [30]:
X_train_w2v = copy.copy(X_train)
X_test_w2v = copy.copy(X_test)

In [35]:
X_train_w2v.head(2)

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,44956a24,1,2,15706,320,50,1722,0,35,-1,79
1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,711ee120,1,0,15704,320,50,1722,0,35,100084,79


In [36]:
for i in names:
    X_train_w2v[i]=X_train_w2v[i].astype('category')
    X_train_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in X_train_w2v[i].cat.categories]

In [37]:
X_train_w2v.head(2)

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,Feature C1 1005,Feature banner_pos 0,Feature site_id 1fbe01fe,Feature site_domain f3845767,Feature site_category 28905ebd,Feature app_id ecad2386,Feature app_domain 7801e8d9,Feature app_category 07d7df22,Feature device_id a99f214a,Feature device_model 44956a24,Feature device_type 1,Feature device_conn_type 2,Feature C14 15706,Feature C15 320,Feature C16 50,Feature C17 1722,Feature C18 0,Feature C19 35,Feature C20 -1,Feature C21 79
1,Feature C1 1005,Feature banner_pos 0,Feature site_id 1fbe01fe,Feature site_domain f3845767,Feature site_category 28905ebd,Feature app_id ecad2386,Feature app_domain 7801e8d9,Feature app_category 07d7df22,Feature device_id a99f214a,Feature device_model 711ee120,Feature device_type 1,Feature device_conn_type 0,Feature C14 15704,Feature C15 320,Feature C16 50,Feature C17 1722,Feature C18 0,Feature C19 35,Feature C20 100084,Feature C21 79


In [38]:
for i in names:
    X_test_w2v[i]=X_test_w2v[i].astype('category')
    X_test_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in X_test_w2v[i].cat.categories]
X_train_w2v = X_train_w2v.values
X_test_w2v = X_test_w2v.values

In [39]:
x_w2v_train = np.random.random((len(X_train_w2v),size*X_train_w2v.shape[1]))

In [ ]:
for j in range(X_train_w2v.shape[1]):
    for i in range(X_train_w2v.shape[0]):
        if X_train_w2v[i,j] in w2v.wv:
            x_w2v_train[i,j*size:(j+1)*size] = w2v.wv[X_train_w2v[i,j]]

In [42]:
x_w2v_test = np.random.random((len(X_test_w2v),size*X_test_w2v.shape[1]))
for j in range(X_test_w2v.shape[1]):
    for i in range(X_test_w2v.shape[0]):
        if X_test_w2v[i,j] in w2v.wv:
            x_w2v_test[i,j*size:(j+1)*size] = w2v.wv[X_test_w2v[i,j]]

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
l = LogisticRegression()
r = RandomForestClassifier(n_estimators=25,max_depth=10)

In [44]:
from sklearn.metrics import log_loss

In [45]:
l.fit(x_w2v_train,y_train)
y_pred = l.predict_proba(x_w2v_test)
print(log_loss(y_test,y_pred))

0.39674377914887066


In [46]:
r.fit(x_w2v_train,y_train)
y_pred = r.predict_proba(x_w2v_test)
print(log_loss(y_test,y_pred))

0.3867947255517792
